In [1]:
import os,traceback
from datetime import datetime
from shutil import copyfile, move
from PIL import Image
from PIL.ExifTags import Base
from pillow_heif import HeifImagePlugin

In [21]:
a = None
b = datetime(2024,1,1)
if a:
    print(a)
else:
    print(b)

2024-01-01 00:00:00


In [ ]:
p = "/Volumes/PS2000/imm_restore/Pictures/(.jpg) Jpeg 图像 (001)"
with os.scandir(p) as entries:
    for entry in entries:
        print(os.path.join(p,entry.name))


In [2]:
source_dir=r"G:\imm_restore\Pictures"
target_dir=r"G:\imm_restore\iphone14"
filepath=r"G:\imm_restore\Pictures\(.jpg) Jpeg 图像 (004)\01220.jpg"
try:
    with Image.open(filepath) as img:  
        exif=img.getexif()
        if exif and exif.get(272) and exif[272]=='iPhone 14':
            print(f"{filepath} is taken with {exif[272]}")
        else:
            print(f"{filepath} is not taken with iPhone 14")
except Exception as e:
    traceback.print_exc()

Traceback (most recent call last):
  File "C:\Users\wilso\AppData\Local\Temp\ipykernel_23180\1063638756.py", line 5, in <module>
    with Image.open(filepath) as img:
       ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\wilso\envs\ds\Lib\site-packages\PIL\Image.py", line 3305, in open
    raise UnidentifiedImageError(msg)
PIL.UnidentifiedImageError: cannot identify image file 'G:\\imm_restore\\Pictures\\(.jpg) Jpeg 图像 (004)\\01220.jpg'


In [24]:
for tag,value in exif.items():
    print(Base(tag).name,"(",tag,") :",value)
    if Base(tag)==306:
        print(datetime.strptime(value, '%Y:%m:%d %H:%M:%S')>datetime(2022,1,1))

TileWidth ( 322 ) : 512
TileLength ( 323 ) : 512
GPSInfo ( 34853 ) : 2110
ResolutionUnit ( 296 ) : 2
ExifOffset ( 34665 ) : 238
Make ( 271 ) : Apple
Model ( 272 ) : iPhone XS
Software ( 305 ) : 15.1
Orientation ( 274 ) : 6
DateTime ( 306 ) : 2022:06:18 12:49:29
True
XResolution ( 282 ) : 72.0
YResolution ( 283 ) : 72.0
HostComputer ( 316 ) : iPhone XS


In [ ]:
import os
from datetime import datetime
from shutil import copyfile, move
from PIL import Image
from PIL.ExifTags import Base
from pillow_heif import HeifImagePlugin

source_dir=r"/Volumes/PS2000/imm_restore/Pictures"
target_dir=r"/Volumes/PS2000/imm_restore/xs"
count = 0
hit_files = 0
for roots,dirs,files in os.walk(source_dir, topdown=False):
    for name in files:
        filepath=os.path.join(roots,name)
        count += 1
        if count > 2000:
            break
        else:
            try:
                with Image.open(filepath,'r') as img:
                    exif=img.getexif()
                    if exif and exif.get(272)=='iPhone 14':
                        hit_files += 1
                        img.close()
                        # copyfile(filepath, os.path.join(target_dir, name))
                        print(f"scanned {count} files, copied #{hit_files}: {filepath}")
            except:
                print(f"read error: {filepath}")


In [116]:
import ffmpeg
from datetime import datetime

# videofile = "/Volumes/PS2000/imm_restore/Videos/MP4-2/02022.mp4"
# videofile = r"/Volumes/PS2000/imm_restore/Videos/MOV8/08021.mov"
videofile = "/Volumes/PS2000/imm_restore/Videos/M4V/00026.m4v"

def deep_traversal(obj, level):
   if isinstance(obj, dict):
      print("")
      level += 1
      for key,value in obj.items():
         print('    '*level + f"{key}: ", end='')
         deep_traversal(value, level)
   elif isinstance(obj, list):
      level += 1
      for item in obj:
         deep_traversal(item, level)
   else:
      print(obj)

def deep_get(obj, key):
   if key in obj:
      return obj[key]
   for k,v in obj.items(): 
      if isinstance(v, dict):
         item = deep_get(v, key)
         if item is not None:
            return item
   return None

def find_video_date(meta):
   mask = "%Y-%m-%dT%H:%M:%S"
   date0 = datetime.strptime(deep_get(meta, 'com.apple.quicktime.creationdate')[:19], mask) if deep_get(meta, 'com.apple.quicktime.creationdate') else datetime(9999,12,31,0,0,0)
   date1 = datetime.strptime(deep_get(meta, 'creation_time')[:19], mask) if deep_get(metadata, 'creation_time') else datetime(9999,12,31,0,0,0)
   date2 = datetime.strptime(deep_get(meta, 'date')[:19], mask) if deep_get(metadata, 'date') else datetime(9999,12,31,0,0,0)
   date = min([date0, date1, date2]) if min([date0, date1, date2]) != datetime(9999,12,31,0,0,0) else None
   return date

metadata = ffmpeg.probe(videofile)

for k,v in metadata.items():
   print(f"{k}: ", end='')
   deep_traversal(v, 0)

model = deep_get(metadata, 'com.apple.quicktime.model')
if model:
   print(model)
else:
   print("Model not found")

date = find_video_date(metadata)
print(date)

Model not found
2019-08-19 17:30:57


In [2]:
import ffmpeg, warnings
from PIL import Image
from pillow_heif import HeifImagePlugin

v1 = r"/Volumes/PS2000/imm_restore/Videos/MOV8/08021.mov"
v2 = r'/Volumes/PS2000/imm_restore/Videos/MOV/00350.mov'
v3 = r'/Volumes/PS2000/imm_restore/Videos/M4V/00000.m4v'
pic1 = r'/Volumes/PS2000/imm_restore/Pictures/(.heic) Heif-Heic 图像 (004)/04000.heic'
pic2 = r'/Volumes/PS2000/imm_restore/Pictures/(.jpg) Jpeg 图像 (001)/01000.jpg'

def is_image(filepath, showerr=True):
    with warnings.catch_warnings(record=True) as w:
        try:
            with Image.open(filepath):
                return True
        except Exception as img_open_error:
            if showerr:
                print(f"Error [{filepath}]: {img_open_error}")
                if w:
                    print(f"Warnings [{filepath}]: {w[0].message}")
            return False

def is_video(filepath, showerr=True):
    if is_image(filepath, False):
        return False
    else:
        try:
            ffmpeg.probe(filepath)
            return True
        except Exception as video_probe_err:
            if showerr:
                print(f"Error [{filepath}]: {video_probe_err}")
            return False

# is_video(v2, True)
# ffmpeg.probe(v3)

# with Image.open(videofile, 'r') as v:
#     print(v.info)
# print(magic.from_file(pic1, mime=True))
# print(magic.from_file(pic2, mime=True))
# print(magic.from_file(videofile, mime=True))
p = r'/Users/wilson/Desktop/xs/01372.jpg'
v = r'/Users/wilson/Pictures/IMG_0227.MOV'
is_video(v, False)
ffmpeg.probe(v)

{'streams': [{'index': 0,
   'codec_name': 'hevc',
   'codec_long_name': 'H.265 / HEVC (High Efficiency Video Coding)',
   'profile': 'Main',
   'codec_type': 'video',
   'codec_tag_string': 'hvc1',
   'codec_tag': '0x31637668',
   'width': 1920,
   'height': 1440,
   'coded_width': 1920,
   'coded_height': 1440,
   'closed_captions': 0,
   'film_grain': 0,
   'has_b_frames': 2,
   'pix_fmt': 'yuvj420p',
   'level': 150,
   'color_range': 'pc',
   'color_space': 'smpte170m',
   'color_transfer': 'bt709',
   'color_primaries': 'smpte432',
   'chroma_location': 'left',
   'refs': 1,
   'id': '0x1',
   'r_frame_rate': '30000/1001',
   'avg_frame_rate': '3600/127',
   'time_base': '1/600',
   'start_pts': 0,
   'start_time': '0.000000',
   'duration_ts': 1397,
   'duration': '2.328333',
   'bit_rate': '11892293',
   'nb_frames': '66',
   'extradata_size': 104,
   'disposition': {'default': 1,
    'dub': 0,
    'original': 0,
    'comment': 0,
    'lyrics': 0,
    'karaoke': 0,
    'forced'

In [3]:
import exiftool
from datetime import datetime
exepath = "/Users/wilson/Envs/d-science/exiftool/exiftool"
pic = r"/Users/wilson/Desktop/xs/01259.jpg"
pic2 = r"/Users/wilson/Pictures/IMG_7128.jpg"
mov = r"/Users/wilson/Pictures/HFON4185 1.MP4"
mov1= r"/Volumes/PS2000/imm_restore/Videos/M4V/00000.m4v"
model = "iPhone XS"

def match_model(metadata, model):
    if ('EXIF:Model' in metadata and model in metadata['EXIF:Model']) or ('QuickTime:Model' in metadata and model in metadata['QuickTime:Model']):
        return True
    else:
        return False
        
def get_create_date(metadata):
    date_mask = "%Y:%m:%d %H:%M:%S"
    if metadata:
        if 'EXIF:CreateDate' in metadata and metadata['EXIF:CreateDate']:
            d0 = datetime.strptime(metadata['EXIF:CreateDate'][:19], date_mask)
        else:
            d0 = datetime(9999,12,31,0,0,0)
        if 'QuickTime:CreateDate' in metadata and metadata['QuickTime:CreateDate']:
            d1 = datetime.strptime(metadata['QuickTime:CreateDate'][:19], date_mask)
        else:
            d1 = datetime(9999,12,31,0,0,0)
        if 'QuickTime:MediaCreateDate' in metadata and metadata['QuickTime:MediaCreateDate']:
            d2 = datetime.strptime(metadata['QuickTime:MediaCreateDate'][:19], date_mask)
        else:
            d2 = datetime(9999,12,31,0,0,0)
        if 'QuickTime:ContentCreateDate' in metadata and metadata['QuickTime:ContentCreateDate']:
            d3 = datetime.strptime(metadata['QuickTime:ContentCreateDate'][:19], date_mask)
        else:
            d3 = datetime(9999,12,31,0,0,0)
        date = min([d0,d1,d2,d3]) if min([d0,d1,d2,d3]) != datetime(9999,12,31,0,0,0) else None
        return date
    else:
        return None
        

with exiftool.ExifToolHelper(executable=exepath) as et:
    exif = et.get_metadata(pic2)
    meta = et.get_metadata(mov, ['-ee'])
    finfo = et.get_metadata('/Users/wilson/Desktop/favorites_9_5_23.html')
print(exif[0])
m = match_model(exif[0], 'iPhone 12')
print(m)
print(f"EXIF:CreateDate: {exif[0]['EXIF:CreateDate']}")

# print(f"CreationDate: {exif[0]['EXIF:Creat']}")
# if ('EXIF:Model' in exif[0] and model in exif[0]['EXIF:Model']) or ('EXIF:LensModel' in exif[0] and model in exif[0]['EXIF:LensModel']):
#     print("matched!")
# else:
#     print("not matched...")
# # if 'iPhone XS' in meta[0]['QuickTime:Model'] or ('Composite:LensID' in meta[0].keys() and 'iPhone XS' in meta[0]['Composite:LensID']):
# #     print(meta[0]['QuickTime:Model'])
print(meta[0])
# print(meta[0]['EXIF:LensModel'])
# print(meta[0]['QuickTime:Model'])
# print(meta[0]['File:MIMEType'].split('/')[0])
print(f"QuickTime:CreateDate:        {meta[0]['QuickTime:CreateDate']}")
print(f"QuickTime:MediaCreateDate:   {meta[0]['QuickTime:MediaCreateDate']}")
# print(f"QuickTime:ContentCreateDate: {meta[0]['QuickTime:ContentCreateDate']}")

d = get_create_date(meta[0])
print(d)

{'SourceFile': '/Users/wilson/Pictures/IMG_7128.jpg', 'ExifTool:ExifToolVersion': 12.73, 'File:FileName': 'IMG_7128.jpg', 'File:Directory': '/Users/wilson/Pictures', 'File:FileSize': 3021069, 'File:FileModifyDate': '2018:07:31 22:11:00+08:00', 'File:FileAccessDate': '2022:10:20 16:22:41+08:00', 'File:FileInodeChangeDate': '2018:08:10 14:51:38+08:00', 'File:FilePermissions': 100644, 'File:FileType': 'JPEG', 'File:FileTypeExtension': 'JPG', 'File:MIMEType': 'image/jpeg', 'File:ExifByteOrder': 'MM', 'File:CurrentIPTCDigest': 'd41d8cd98f00b204e9800998ecf8427e', 'File:ImageWidth': 4752, 'File:ImageHeight': 3168, 'File:EncodingProcess': 0, 'File:BitsPerSample': 8, 'File:ColorComponents': 3, 'File:YCbCrSubSampling': '2 2', 'JFIF:JFIFVersion': '1 1', 'JFIF:ResolutionUnit': 0, 'JFIF:XResolution': 72, 'JFIF:YResolution': 72, 'EXIF:Make': 'Canon', 'EXIF:Model': 'Canon EOS 500D', 'EXIF:XResolution': 72, 'EXIF:YResolution': 72, 'EXIF:ResolutionUnit': 2, 'EXIF:ModifyDate': '2018:07:31 15:24:37', 'EX

In [1]:
import mimetypes, ffmpeg
from PIL import Image
p = r'/Users/wilson/Desktop/xs/01372.jpg'
v = r'/Users/wilson/Pictures/TJJM8974.MOV'
t, _ = mimetypes.guess_type(p)
print(t)
# try:
#     with Image.open(v, "r") as img:
#         e = img.getexif()
# except Exception as er:
#     print(er)
ffmpeg.probe(v)

image/jpeg


{'streams': [{'index': 0,
   'codec_name': 'hevc',
   'codec_long_name': 'H.265 / HEVC (High Efficiency Video Coding)',
   'profile': 'Main',
   'codec_type': 'video',
   'codec_tag_string': 'hvc1',
   'codec_tag': '0x31637668',
   'width': 1080,
   'height': 1920,
   'coded_width': 1088,
   'coded_height': 1920,
   'closed_captions': 0,
   'film_grain': 0,
   'has_b_frames': 0,
   'pix_fmt': 'yuv420p',
   'level': 153,
   'color_range': 'tv',
   'color_space': 'bt709',
   'color_transfer': 'bt709',
   'color_primaries': 'bt709',
   'chroma_location': 'left',
   'refs': 1,
   'id': '0x1',
   'r_frame_rate': '30/1',
   'avg_frame_rate': '1092/41',
   'time_base': '1/19200',
   'start_pts': 0,
   'start_time': '0.000000',
   'duration_ts': 131200,
   'duration': '6.833333',
   'bit_rate': '4239117',
   'nb_frames': '182',
   'extradata_size': 110,
   'disposition': {'default': 1,
    'dub': 0,
    'original': 0,
    'comment': 0,
    'lyrics': 0,
    'karaoke': 0,
    'forced': 0,
    'h